In [1]:
import cv2

%matplotlib inline

Fontconfig warning: ignoring UTF-8: not a valid region tag


In [2]:
img = cv2.imread("chars.jpeg")

In [3]:
import matplotlib.pyplot as plt

In [12]:
import gym
import numpy as np

In [28]:
def value_iteration(v_inp, gamma, env: gym.Env):
    ret = np.zeros(env.observation_space.n)
    for sid in range(env.observation_space.n):
        temp_v = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            dest_state, reward, terminated, truncated, info = env.step(action)
            prob = info["prob"]
            temp_v[action] += prob * (reward + gamma * v_inp[dest_state] * (not terminated))
        ret[sid] = np.max(temp_v)
    return ret

def greed_policy(v_inp, gamma, env: gym.Env):
    ret = np.zeros(env.observation_space.n)
    for sid in range(env.observation_space.n):
        temp_v = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            dest_state, reward, terminated, truncated, info = env.step(action)
            prob = info["prob"]
            temp_v[action] += prob * (reward + gamma* v_inp[dest_state] * (not terminated))
        ret[sid] = np.argmax(temp_v)
    return ret

In [31]:
env = gym.make("Taxi-v3")
gamma = 0.9
cum_reward = 0
n_rounds = 500
for epoch in range(n_rounds):
    observation = env.reset()
    v = np.zeros(env.observation_space.n)

    # solve MDP
    for _ in range(100):
        v_old = v.copy()
        v = value_iteration(v_old, gamma, env)
        if np.all(v == v_old):
            break
    print(v)
    policy = greed_policy(v, gamma, env)
    print(policy)

[-9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439
 -9.99973439 -9.99973439 -9.99973439 -9.99973439 -9

TypeError: can't multiply sequence by non-int of type 'float'

In [16]:
observation = env.reset()

In [25]:
env.action_space.n

6

In [21]:
env.step()

(266,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)})